In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Prompt_Semantic"
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Prompt_Semantic
/content/drive/MyDrive/Prompt_Semantic


In [3]:
# Set up the environment
!pip install -r requirements.txt


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-uj5bje7b
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-uj5bje7b
  Resolved https://github.com/huggingface/transformers to commit 746104ba6f0514159d58dc2fb09c887d0e9d4863
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.5 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9614041 sha256=b8647a66dc2c6b2de8b74ab5227defb6b54b896c773214cff3e2be98f180ecae
  Stored in directory: /tmp/pip-ephem-wheel-cache-q73sdd4d/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully b

In [7]:
# !pip install transformers accelerate bitsandbytes>0.37.0

In [ ]:
# !python main.py \
#   --train_date_path


In [4]:
train_path = "/content/drive/MyDrive/Prompt_Semantic/Data/karma-version/sources/"
test_path = "/content/drive/MyDrive/Prompt_Semantic/Data/karma-version/models-json/"
ontology_path = "/content/drive/MyDrive/Prompt_Semantic/ontologies/cidoc-crm-v6_2.rdf"

In [5]:

from data_loader import generate_data
from prompt_generator import load_ontology, generate_prompt_without_semantic_typing
train_data, test_data = generate_data(train_path, test_path,  3, 2)
ontology = load_ontology(ontology_path)
prompt = generate_prompt_without_semantic_typing(train_data, ontology, True)

Loading data from [0, 1] files
Loading data from /content/drive/MyDrive/Prompt_Semantic/Data/karma-version/sources/s01-cb.json and /content/drive/MyDrive/Prompt_Semantic/Data/karma-version/models-json/s01-cb-model.json
Loading data from /content/drive/MyDrive/Prompt_Semantic/Data/karma-version/sources/s02-dma.json and /content/drive/MyDrive/Prompt_Semantic/Data/karma-version/models-json/s02-dma-model.json
Loading data from [2] files
Loading data from /content/drive/MyDrive/Prompt_Semantic/Data/karma-version/sources/s03-ima-artists.json and /content/drive/MyDrive/Prompt_Semantic/Data/karma-version/models-json/s03-ima-artists-model.json


In [6]:
for i, data in enumerate(test_data):
        prompt += f"Table: "
        for j, table in enumerate(data['table']):
            prompt += f"{table}\n"
        prompt += f"The semantic graph of the table in the strict format of given examples above:"

In [7]:
text_file = open("Output.txt", "w")

text_file.write(prompt)

text_file.close()

In [ ]:
from openai import OpenAI

model = OpenAI(api_key="YourAPIKey", base_url="https://api.deepseek.com")

input_msg = [{"role": "user", "content": prompt}]
response = model.chat.completions.create(
    model="deepseek-chat",
    messages=input_msg,
    stream=False
)
response = response.choices[0].message.content

text_file = open("response.txt", "w")

text_file.write(response)

text_file.close()

In [14]:
import json

def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def mean_reciprocal_rank(predicted, gold):
    ranks = []
    for item in gold:
        if item in predicted:
            rank = predicted.index(item) + 1
            ranks.append(1 / rank)
    return sum(ranks) / len(gold) if ranks else 0

def find_differences(list1, list2):
    set1 = set(tuple(item) for item in list1)
    set2 = set(tuple(item) for item in list2)
    differences = set1.symmetric_difference(set2)
    return [list(item) for item in differences]

# Load the JSON files
gold_semantics= load_json('Data/models-triples/s05-met-model.json')['semantic_triples']
predicted_semantics = load_json('s03_predict.json')['SetSemanticType']  # Replace with the actual path to the other file

# Calculate MRR
mrr = mean_reciprocal_rank(predicted_semantics, gold_semantics)
print(f"Mean Reciprocal Rank (MRR): {mrr}")

# Find differences
differences = find_differences(predicted_semantics, gold_semantics)
print("Differences between the triples:")
for diff in differences:
    print(diff)

Mean Reciprocal Rank (MRR): 0.16166817788334814
Differences between the triples:
['crm:E33_Linguistic_Object3', 'rdf:value', 'Credit Line']
['crm:E33_Linguistic_Object3', 'rdf:value', 'Provenance']
['crm:E33_Linguistic_Object4', 'rdf:value', 'Description']
['crm:E33_Linguistic_Object5', 'rdf:value', 'Provenance']
['crm:E33_Linguistic_Object2', 'rdf:value', 'Dimensions']
['crm:E33_Linguistic_Object1', 'rdf:value', 'Credit Line']
['crm:E54_Dimension1', 'rdf:value', 'Dimensions']
['crm:E33_Linguistic_Object2', 'rdf:value', 'Description']
['crm:E33_Linguistic_Object1', 'rdf:value', 'Medium']
['crm:E57_Material1', 'rdfs:label', 'Medium']


In [15]:
predicted_semantics.sort(key=lambda x: x[2])
gold_semantics.sort(key=lambda x: x[2]  )
predicted_semantics


[['crm:E42_Identifier2', 'rdf:value', 'Accession Number'],
 ['crm:E55_Type1', 'rdfs:label', 'Classification'],
 ['crm:E33_Linguistic_Object1', 'rdf:value', 'Credit Line'],
 ['crm:E52_Time-Span1', 'rdfs:label', 'Date'],
 ['crm:E33_Linguistic_Object2', 'rdf:value', 'Description'],
 ['crm:E54_Dimension1', 'rdf:value', 'Dimensions'],
 ['crm:E53_Place1', 'rdfs:label', 'Geography'],
 ['crm:E57_Material1', 'rdfs:label', 'Medium'],
 ['crm:E33_Linguistic_Object3', 'rdf:value', 'Provenance'],
 ['skos:Concept1', 'skos:prefLabel', 'What'],
 ['crm:E82_Actor_Appellation1', 'rdf:value', 'Who'],
 ['crm:E39_Actor1', 'karma:classLink', 'artist_uri'],
 ['crm:E52_Time-Span1', 'crm:P82a_begin_of_the_begin', 'date_earliest'],
 ['crm:E52_Time-Span1', 'crm:P82b_end_of_the_end', 'date_latest'],
 ['crm:E42_Identifier1', 'rdf:value', 'id'],
 ['crm:E38_Image1', 'karma:classLink', 'image'],
 ['crm:E22_Man-Made_Object1', 'karma:classLink', 'object_uri'],
 ['crm:E12_Production1', 'karma:classLink', 'production_uri']

In [16]:
gold_semantics

[['crm:E42_Identifier2', 'rdf:value', 'Accession Number'],
 ['crm:E55_Type1', 'rdfs:label', 'Classification'],
 ['crm:E33_Linguistic_Object3', 'rdf:value', 'Credit Line'],
 ['crm:E52_Time-Span1', 'rdfs:label', 'Date'],
 ['crm:E33_Linguistic_Object4', 'rdf:value', 'Description'],
 ['crm:E33_Linguistic_Object2', 'rdf:value', 'Dimensions'],
 ['crm:E53_Place1', 'rdfs:label', 'Geography'],
 ['crm:E33_Linguistic_Object1', 'rdf:value', 'Medium'],
 ['crm:E33_Linguistic_Object5', 'rdf:value', 'Provenance'],
 ['skos:Concept1', 'skos:prefLabel', 'What'],
 ['crm:E82_Actor_Appellation1', 'rdf:value', 'Who'],
 ['crm:E39_Actor1', 'karma:classLink', 'artist_uri'],
 ['crm:E52_Time-Span1', 'crm:P82a_begin_of_the_begin', 'date_earliest'],
 ['crm:E52_Time-Span1', 'crm:P82b_end_of_the_end', 'date_latest'],
 ['crm:E42_Identifier1', 'rdf:value', 'id'],
 ['crm:E38_Image1', 'karma:classLink', 'image'],
 ['crm:E22_Man-Made_Object1', 'karma:classLink', 'object_uri'],
 ['crm:E12_Production1', 'karma:classLink', '

In [ ]:
    {
        "id":10000831,
        "image":"http:\/\/images.metmuseum.org\/CRDImages\/ad\/web-large\/DP104414.jpg",
        "Date":"1700-1710",
        "date_latest":"",
        "date_earliest":"",
        "Geography":"Mid-Atlantic, New York City, New York, United States",
        "Medium":"Silver",
        "Dimensions":"\n",
        "Classification":"Silver",
        "Credit Line":"Samuel D. Lee Fund, 1938",
        "Accession Number":"38.63",
        "object_uri":"http:\/\/www.metmuseum.org\/object\/38.63",
        "type_assignment_uri":"http:\/\/www.metmuseum.org\/object\/38.63\/work_type",
        "production_uri":"http:\/\/www.metmuseum.org\/object\/38.63\/production",
        "Who":"Cornelius Kierstede",
        "artist_uri":"http:\/\/www.metmuseum.org\/person\/cornelius-kierstede",
        "What":[
            "Metal",
            "Silver",
            "Bowls"
        ],
        "Description":"Derived from Dutch, Scandinavian, and English sources, brandywine bowls were used on festive occasions such as the kindermaal, when women gathered to welcome a newborn child. Following Dutch custom, a bowl would be filled with raisins and brandy and passed from guest to guest. Here, boldly chased tulips create a sumptuous effect. The bowl belonged to Theunis Jacobsen Quick, a wealthy baker, and his wife, Vroutje 4516 Janse Haring.",
        "Provenance":"\n"
    },